In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
file_list = drive.ListFile({'q': "'1XzHs2EsiefV9pIBaB0BkOuNOGRHtnB4u' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: random.csv, id: 12fYvpkZL_S8V7ETquEHdG1bGf3qi2rlg
title: sample_submission.csv, id: 1_HxDS3W0BvAxpEom2GGAHBG2Q0b-zl81
title: train.csv, id: 1WSB3nMD2zaOGn90pm-qX9Y77qb-S88AX
title: test.csv, id: 16U7xU_UJ9hiQ0EFK-bk1aFa0kWuCNPDC


In [0]:

import pandas as pd
import numpy as np

sample_download = drive.CreateFile({'id':'1_HxDS3W0BvAxpEom2GGAHBG2Q0b-zl81'})
sample_download.GetContentFile('sample_submission.csv')
df_train = drive.CreateFile({'id':'1WSB3nMD2zaOGn90pm-qX9Y77qb-S88AX'})
df_train.GetContentFile('train.csv')
df_test = drive.CreateFile({'id':'16U7xU_UJ9hiQ0EFK-bk1aFa0kWuCNPDC'})
df_test.GetContentFile('test.csv')

In [0]:
from sklearn.preprocessing import LabelEncoder # can use something else also
import math
import os
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [0]:
import pandas
import random
import sys 



train_count = 29118021
test_count = 8607230
r_train =int(train_count/1.2)

r_test =int(test_count/1.2)


skip_training = sorted(random.sample(range(train_count),(train_count-r_train)))
skip_testing = sorted(random.sample(range(test_count),(test_count-r_test)))

                      
df_train = pd.read_csv('train.csv', skiprows=skip_training, header=0, names = ["row_id", "x", "y", "accuracy", "time", "place_id"])

df_testing = pd.read_csv('test.csv', skiprows=skip_testing, header=0,names = ["row_id", "x", "y", "accuracy", "time"])
#  

In [0]:
def time_preprocessing(df):
    intialdate = np.datetime64('2018-01-01T01:01', dtype='datetime64[m]')  
    time = pd.DatetimeIndex(df['time'] + intialdate)       
    df['minute'] = time.minute
    df['hour'] = time.hour + time.minute / 60.0
    df['weekday'] = time.weekday + time.hour / 24.0
    df['day'] = time.dayofyear
    df['month'] = time.month - 1
    df['year'] = time.year - 2018 + time.month / 12.0   
    return df

df_train = time_preprocessing(df_train)
df_testing = time_preprocessing(df_testing)


In [0]:

def create_cell(df, xcell_no, ycell_no):
    sizeofx = 10.0 / xcell_no
    sizeofy = 10.0 / ycell_no
    pos_x = ((df['x'].values) / sizeofx).astype(int)
    pos_y = ((df['y'].values ) / sizeofy).astype(int)
    df['grid_cell'] = pos_y * xcell_no + pos_x
    return df
  


In [0]:
xcell_no = 50
ycell_no = 80 
cell_for_validation = 30
n_cells = xcell_no * ycell_no
df_train = create_cell(df_train, xcell_no, ycell_no)
df_testing = create_cell(df_testing, xcell_no, ycell_no)


In [0]:
import math
def accuracy_preprocessing(df):     #CREATED LOG OF ACCURACY
    df['log_accuracy'] = np.log10(df.accuracy.values)
#     print(df['log_accuracy'])
    return df

df_train = accuracy_preprocessing(df_train)
df_testing = accuracy_preprocessing(df_testing)

In [0]:
def train_test_split(df, cell_id):
    
    df_train_cell = df.loc[df.grid_cell == cell_id]
    time = df_train_cell.time.quantile(0.75) 
    
    maxy = df_train_cell.y.max()
    maxx=df_train_cell.x.max()
    miny=df_train_cell.y.min()
    minx=df_train_cell.x.min()

    X_train = df.loc[(df.x > minx)&(df.x < maxx)&(df.y > miny)&(df.y < maxy)&(df.time < time)]
    
    place_count = X_train.place_id.value_counts()
    threshold = place_count.quantile(0.1)
    place_abv_th = (place_count[X_train.place_id] >= threshold).values

    X_train = X_train.loc[place_abv_th]
    y_train = X_train.place_id    
    X_train = X_train.drop(['row_id', 'place_id'], axis=1, inplace=False)

    
    X_validation = df_train_cell[df_train_cell.time > time]
    y_validation = X_validation.place_id
    X_validation = X_validation.drop(['row_id', 'place_id'], axis=1, inplace=False)

    
    return X_train, X_validation, y_train, y_validation

In [0]:
def test_preprocess(df, cell_id):
    df_test_cell = df.loc[df.grid_cell == cell_id]
    time = df_test_cell.time.quantile(0.75) 
    
    maxy = df_test_cell.y.max()
    maxx=df_test_cell.x.max()
    miny=df_test_cell.y.min()
    minx=df_test_cell.x.min()

    X_test = df.loc[(df.x > minx)&(df.x < maxx)&(df.y > miny)&(df.y < maxy)&(df.time < time)]
    X_test = X_test.drop(['row_id'], axis=1, inplace=False)
    
    return X_test

In [0]:
def feature_engineering(df, attribute_weights):
    DF = df.copy()
    for col in attribute_weights:
        DF[col] = DF[col]*attribute_weights[col]
    return DF

In [0]:
import warnings 
warnings.filterwarnings("ignore")
cell_ids = np.random.choice(n_cells, cell_for_validation,replace=True)
print(cell_ids)


[   3  930 1474 1896 2713 1544 2278 3102 2822 2669 3638 1362 1872 2217
 3013  279 3295   87  203  444 1879  541 2879 1030 2912  194 3554 1669
 2511  139]


In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,f1_score

#for accuracy
def KNN_classifier(df_train, df_testing, cell_id, 
                         cols=None, attribute_weights=None, params={}):
    
    
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    X_test =test_preprocess(df_testing, cell_id)
    row_ids = X_test.index
    

    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

    
    clf = KNeighborsClassifier(**params)
    clf.fit(X_train, y_train)
    y_prob_valid = pd.DataFrame(clf.predict_proba(X_valid), columns=clf.classes_)

    
    #for cv data
    predictions = clf.predict(X_valid)
    y_pred3_valid = [] 
    y_pred1_valid= [] 
    for _, row in y_prob_valid.iterrows():
        data_valid = row.sort_values(ascending=False).index[:3]
        y_pred3_valid.append(data_valid)
        y_pred1_valid.append(data_valid[:1])

    
#     for testing data
    y_prob_test = pd.DataFrame(clf.predict_proba(X_test),columns=clf.classes_)
    
    y_pred3_testing = [] 
    y_pred1_testing= [] 
    for _, row in y_prob_test.iterrows():
        data_testing = row.sort_values(ascending=False).index[:3]
        y_pred3_testing.append(data_testing)
        y_pred1_testing.append(data_testing[:1])
        
    return y_pred3_valid, y_pred3_testing, X_train, np.array(y_pred1_valid).reshape(1,-1)[0],y_train, y_valid,f1_score(y_valid,predictions,average="macro"),clf.score(X_valid,y_valid), recall_score(y_valid,predictions,average='macro'), precision_score(y_valid,predictions,average='macro')




In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix

#For finding top 3 places
def KNN_classifier_place_predict(df_train, df_testing, cell_id, 
                         cols=None, attribute_weights=None, params={}):
    
    
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    X_test =test_preprocess(df_testing, cell_id)
    row_ids = X_test.index

    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

    #label encoder changes
    le = LabelEncoder()
    y_valid = le.fit_transform(y_valid)
    y_train = le.fit_transform(y_train)

    
    # Creating KNN classifier
    clf = KNeighborsClassifier(**params)
    clf.fit(X_train, y_train)

    y_pred_test = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred_test, axis=1)[:,::-1][:,:3])
    return  row_ids, pred_labels


In [0]:
  from sklearn.metrics import precision_score, recall_score, accuracy_score
  

  y_valids = []
  score12=[]
  recallArr=[]
  precisionArr=[]
  f_1=[]
  attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}
  params = {'n_neighbors':25, 'weights':'distance', 'metric':'manhattan'}
  
#   prediction = np.zeros((len(df_testing), 3), dtype=int)
  
  for cell_id in cell_ids:
    y_pred3_valid, y_pred3_testing, X_train, y_pred1_valid, y_train, y_valid,f1Score,score1,recall1,precision1 = KNN_classifier(df_train, df_testing, cell_id, cols=attribute_weights.keys(), attribute_weights=attribute_weights,params=params)
    f_1.append(f1Score)
    y_valids.append(y_valid)
    precisionArr.append(precision1)
    recallArr.append(recall1)
    score12.append(score1*100)
    
  
  print("Top 3 places in each cell")
#   print(KNN_pred_test)
  print("Precision")
  print(np.mean(precisionArr))
  print("Recall")
  print(np.mean(recallArr))
  print("Recall")
  print(np.mean(f_1))
  print ("Accuracy of KNN in {} cells:  {:.4f}".format(cell_for_validation,np.mean(score12)))

Top 3 places in each cell
Precision
0.0011842837715301457
Recall
0.006442664453488708
Recall
0.0012427539994896073
Accuracy of KNN in 30 cells:  6.9970


In [0]:


prediction = np.zeros((len(df_testing), 3), dtype=int)
for cell_id in cell_ids:
  row_ids, pred_labels = KNN_classifier_place_predict(df_train, df_testing, cell_id, cols=attribute_weights.keys(), attribute_weights=attribute_weights,params=params)
  prediction[row_ids] = pred_labels 
top3 = pd.DataFrame(prediction, dtype=str, columns=['place1', 'place2', 'place3'])

NameError: ignored

In [0]:
top3 = top3[(top3.place1 != '0')]
top3

,place1,place2,place3
9,9111475485,3894834079,6980929490
20,7598046358,9921906896,3395406015
120,8046080829,8424529442,9204434432
285,2369577266,6347016508,9012687742
549,4028357018,5802767610,5441560478
568,8772469670,8809927909,8401678699
620,2509729209,4930245341,1572122584
697,1158252651,9834128527,3630198387
1346,8766814189,4278088115,5946611563
1460,3404427129,4944115081,2920348575


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix


def RF_classifier(df_train, cell_id, 
                         attribute_weights=None,cols=None, params={}):

    
    # Split data by time
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    

    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

   
    # Creating RandomForest classifier
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    y_prob = pd.DataFrame(clf.predict_proba(X_valid), columns=clf.classes_)
#     y_prob = clf.predict_proba(X_valid)
    predictions = clf.predict(X_valid)
    y_pred3 = [] 
    y_pred1 = []
    for _, row in y_prob.iterrows():
        data = row.sort_values(ascending=False).index[:3]
        y_pred3.append(data)
        y_pred1.append(data[:1])
    
    return y_pred3, np.array(y_pred1).reshape(1,-1)[0], X_valid, y_valid, X_train, y_train,f1_score(y_valid,predictions,average="macro"), clf.score(X_valid,y_valid),recall_score(y_valid,predictions,average='macro'),precision_score(y_valid,predictions,average='macro')




In [0]:
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

f_1=[]
y_valids = []
score12=[]
conf22=[]
recallArr=[]
precisionArr=[]
# Randomly select cells for validating
# for cell_id in tqdm(cell_ids, total=cell_for_validation):
params = {'n_estimators':20, 'criterion':'gini', 'max_features':'auto'}
attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}
for cell_id in cell_ids:

  y_pred3, y_pred1, X_valid_rf, y_valid_rf, X_train_rf, y_train_rf,f1score ,score1,recall1,precision1 = RF_classifier(df_train, cell_id,attribute_weights=attribute_weights, cols=attribute_weights.keys(),params=params)
  f_1.append(f1score)
  precisionArr.append(precision1)
  recallArr.append(recall1)
  score12.append(score1*100)


print("Presision:")
print(np.mean(precisionArr))
print("Recall:")
print(np.mean(recallArr))
print("f1 score:")
print(np.mean(f_1))
print ("Accuracy of RandomForest in {} cells:  {:.4f}".format(cell_for_validation,score1*100))





  
  

Presision:
0.059271560304358525
Recall:
0.06468989141765738
f1 score:
0.05321650584503949
Accuracy of RandomForest in 30 cells:  41.9378


In [0]:
# #ROC


'''from collections import defaultdict
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

X_roc = np.concatenate((X_train_rf, X_valid_rf))
X_rocdf = pd.DataFrame(data=X_roc)

y_roc = np.concatenate((y_train_rf, y_valid_rf))
y_rocdf = pd.DataFrame(data=y_roc)
# y_rocdf = pd.DataFrame(data=y_roc)
y_rocdf = label_binarize(y_rocdf, classes=np.unique(y_roc))


'''

In [0]:
'''from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm, datasets

n_classes = y_rocdf.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X_rocdf, y_rocdf, test_size=0.5,
                                                    random_state=0)


#clf=RandomForestClassifier(n_estimators=20, criterion='gini', max_features='auto')
clf = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))'''

In [0]:
#y_score =clf.fit(X_train, y_train).decision_function(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))


In [0]:
'''fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[i], y_score[i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
#Plot of a ROC curve for a specific class
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()'''

In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix


def RF_classifier_place_predict(df_train, df_testing, cell_id, 
                         cols=None, attribute_weights=None, params={}):
    
    
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    X_test =test_preprocess(df_testing, cell_id)
    row_ids = X_test.index
    
    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

    #label encoder changes
    le = LabelEncoder()
    y_valid = le.fit_transform(y_valid)
    y_train = le.fit_transform(y_train)

    
    # Creating RF classifier
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)

    y_pred_test = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred_test, axis=1)[:,::-1][:,:3])
    return  row_ids, pred_labels


In [0]:
import warnings
warnings.filterwarnings("ignore")

params = {'n_estimators':20, 'criterion':'gini', 'max_features':'auto'}
attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}

prediction = np.zeros((len(df_testing), 3), dtype=int)
for cell_id in cell_ids:
  row_ids, pred_labels = RF_classifier_place_predict(df_train, df_testing, cell_id, cols=attribute_weights.keys(), attribute_weights=attribute_weights,params=params)
  prediction[row_ids] = pred_labels 
top3 = pd.DataFrame(prediction, dtype=str, columns=['place_1', 'place_2', 'place_3'])

In [0]:
top3 = top3[(top3.place_1 != '0')]
top3

,place_1,place_2,place_3
71,3533292414,7545953421,9854732412
213,5877025704,4310925224,5928300593
279,3923639164,1154304908,4139994690
446,4310925224,5877025704,1803377869
623,4982970923,2141925984,4683657558
734,5733726201,6396383243,8337580720
813,2675934532,9137732132,3817827051
901,3803708092,1067384388,2123402564
1105,3329951661,9137732132,3817827051
1533,1131406470,8104175978,5004578735


In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,f1_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix


def AdaBoost_classifier(df_train, cell_id,
                         attribute_weights=None,cols=None, params={}):
    
    # Split data by time
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id )
   
    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

   
    clf = AdaBoostClassifier(**params)
    clf.fit(X_train, y_train)
    predictions=clf.fit(X_train,y_train)
    y_prob = pd.DataFrame(clf.predict_proba(X_valid), columns=clf.classes_)
#     y_prob = clf.predict_proba(X_valid)
    predictions = clf.predict(X_valid)
    y_pred3 = [] 
    y_pred1 = [] 
    for _, row in y_prob.iterrows():
        data = row.sort_values(ascending=False).index[:3]
        y_pred3.append(data)
        y_pred1.append(data[:1])
    
    return y_pred3, np.array(y_pred1).reshape(1,-1)[0], y_valid,clf.score(X_valid,y_valid),recall_score(y_valid,predictions,average='macro'),precision_score(y_valid,predictions,average='macro')




In [0]:
  y_valids = []
  score12=[]
  f_1=[]
  conf22=[]
  recallArr=[]
  precisionArr=[]

  attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}
  params = {'n_estimators':100, 'algorithm':'SAMME.R'}

  for cell_id in cell_ids:

    y_pred3, y_pred1, y_valid,score1,recall1,precision1 = AdaBoost_classifier(df_train, cell_id,attribute_weights=attribute_weights, cols=attribute_weights.keys(),params=params)

#     f_1.append(f1_score)
    precisionArr.append(precision1)
    recallArr.append(recall1)
    score12.append(score1*100)
    y_valids.append(y_valid)
  print("Presision:")  
  print(np.mean(precisionArr))
  print("Recall:")
  print(np.mean(recallArr))
  print ("Accuracy of Adaboost in {} cells:  {:.4f}".format(cell_for_validation,np.mean(score12)))

Presision:
0.0021308586351430564
Recall:
0.009695941768546735
Accuracy of Adaboost in 30 cells:  11.0687


In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier


def ADA_classifier_place_predict(df_train, df_testing, cell_id, 
                         cols=None, attribute_weights=None, params={}):
    
    
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    X_test =test_preprocess(df_testing, cell_id)
    row_ids = X_test.index
    

    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

    #label encoder changes
    le = LabelEncoder()
    y_valid = le.fit_transform(y_valid)
    y_train = le.fit_transform(y_train)

    
    # Creating RF classifier
    clf = AdaBoostClassifier(**params)
    clf.fit(X_train, y_train)

    y_pred_test = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred_test, axis=1)[:,::-1][:,:3])
    return  row_ids, pred_labels


In [0]:
import warnings
warnings.filterwarnings("ignore")

attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}
params = {'n_estimators':100, 'algorithm':'SAMME.R'}

prediction = np.zeros((len(df_testing), 3), dtype=int)
for cell_id in cell_ids:
  row_ids, pred_labels = ADA_classifier_place_predict(df_train, df_testing, cell_id, cols=attribute_weights.keys(), attribute_weights=attribute_weights,params=params)
  prediction[row_ids] = pred_labels 
top3 = pd.DataFrame(prediction, dtype=str, columns=['place_1', 'place_2', 'place_3'])

In [0]:
top3 = top3[(top3.place_1 != '0')]
top3

,place_1,place_2,place_3
65,1849961373,1236757854,1623207946
126,5547911301,8237727800,1741496596
200,1229926477,8348606078,2165725176
381,3529131926,6594040203,5814238588
563,8257893471,2740133705,5218890251
954,4065305600,7106391200,6764829406
981,4054573731,2545313276,2835853703
1016,2634427271,2532262550,5694414078
1184,2634427271,2532262550,5694414078
1305,2030648078,9101875582,5895904429


In [0]:
#used for XGB
def CalculateScoreAvg(score1):  
  count=0
  Avg=0
  for i in range(len(score1)):
    if(score1[i]>0.001):
      count=count+score1[i]
      Avg=Avg+1
  accuracy= count/Avg
  return accuracy

#

In [0]:
from sklearn.metrics import precision_score,f1_score
from sklearn.metrics import recall_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix


def XGB_classifier(df_train, cell_id,
                         cols=None, attribute_weights=None, params={}):
    
    # Split data by time
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
   

    X_train = feature_engineering(X_train, attribute_weights)
    X_valid = feature_engineering(X_valid, attribute_weights)
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
    y_train = y_train.values
    y_valid = y_valid.values  

   
    clf = XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_prob = pd.DataFrame(clf.predict_proba(X_valid), columns=clf.classes_)
    predictions = clf.fit(X_train, y_train)
#     y_prob = clf.predict_proba(X_valid)
    predictions = clf.predict(X_valid)
    y_pred3 = [] 
    y_pred1 = [] 
    for _, row in y_prob.iterrows():
        data = row.sort_values(ascending=False).index[:3]
        y_pred3.append(data)
        y_pred1.append(data)
    
    return y_pred3, np.array(y_pred1).reshape(1,-1)[0], y_valid,clf.score(X_valid,y_valid),recall_score(y_valid,predictions,average='macro'),precision_score(y_valid,predictions,average='macro')




In [0]:
  y_valids = []
  score12=[]
  conf22=[]
  recallArr=[]
  f_1=[]
  precisionArr=[]
  attribute_weights = {'x':350, 'y':950, 'hour':15,'weekday':15,'day':0.25, 'accuracy': 35,'month':4, 'year': 18}
  params = {'n_neighbors':30, 'booster':'gblinear'}

  for cell_id in cell_ids:

    y_pred3, y_pred1, y_valid,score1,recall1,precision1 = XGB_classifier(df_train, cell_id,  cols=attribute_weights.keys(), attribute_weights=attribute_weights,params=params)
    precisionArr.append(precision1)
    recallArr.append(recall1)
    score12.append(score1*100)
    y_valids.append(y_valid)
  
  print("precision")
  print(np.mean(precisionArr))
  print("recall")
  print(np.mean(recallArr))
  print ("Accuracy of XGB in {} cells:  {:.4f}".format(cell_for_validation,CalculateScoreAvg(score12)))

precision
0.0005203644514649397
recall
0.00215144547947801
Accuracy of XGB in 30 cells:  2.5113


In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score,f1_score, recall_score


def GNB_classifier(df_train, cell_id, cols=None):
    # Split data by time
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)

    
    X_train = X_train[cols].values
    X_valid = X_valid[cols].values
    y_train = y_train.values
    y_valid = y_valid.values
    
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    
    return  y_pred, y_valid, clf.score(X_valid,y_valid), f1_score(y_valid,y_pred,average="macro"), recall_score(y_valid,y_pred,average='macro'), precision_score(y_valid,y_pred,average='macro')


In [0]:
# from tqdm import tqdm, tnrange

score1=[]
y_preds = []
y_valids = []
precisionArr = []
recallArr = []
f_1 = []

cols = ['x', 'y', 'year','accuracy','day','hour','month']

for cell_id in cell_ids:
    y_pred, y_valid,score2 ,f1score, recall ,  precision = GNB_classifier(df_train, cell_id, cols=cols)
    score1.append(score2*100)
    y_valids.append(y_valid)
    f_1.append(f1score)
    precisionArr.append(precision)
    recallArr.append(recall)
    
    
print("precision")
print(np.mean(precisionArr))
print("recall")
print(np.mean(recallArr))
print("f1 Score")
print(np.mean(f_1))
print("ACCURACY IS:")
print(np.mean(score1))

precision
0.05067175445404625
recall
0.07256365783879612
f1 Score
0.050208469339175846
ACCURACY IS:
33.26559369462925


In [0]:
from sklearn.naive_bayes import GaussianNB


def GNB_classifier_place_predict(df_train, df_testing, cell_id, cols=None):
    
    X_train, X_valid, y_train, y_valid = train_test_split(df_train, cell_id)
    X_test =test_preprocess(df_testing, cell_id)
    row_ids = X_test.index
    
    X_train = X_train[X_train.columns.values].values
    X_valid = X_valid[X_valid.columns.values].values
      
    y_train = y_train.values
    y_valid = y_valid.values  

    #label encoder changes
    le = LabelEncoder()
    y_valid = le.fit_transform(y_valid)
    y_train = le.fit_transform(y_train)

    
    # Creating RF classifier
    clf = GaussianNB()
    clf.fit(X_train, y_train)

    y_pred_test = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred_test, axis=1)[:,::-1][:,:3])
    return  row_ids, pred_labels


In [0]:
import warnings
warnings.filterwarnings("ignore")



prediction = np.zeros((len(df_testing), 3), dtype=int)
for cell_id in cell_ids:
  row_ids, pred_labels = GNB_classifier_place_predict(df_train, df_testing, cell_id)
  prediction[row_ids] = pred_labels 
top3 = pd.DataFrame(prediction, dtype=str, columns=['place_1', 'place_2', 'place_3'])

In [0]:
top3 = top3[(top3.place_1 != '0')]
top3

,place_1,place_2,place_3
65,1849961373,1236757854,1623207946
126,5547911301,8237727800,1741496596
200,1229926477,8348606078,2165725176
381,3529131926,6594040203,5814238588
563,8257893471,2740133705,5218890251
954,4065305600,7106391200,6764829406
981,4054573731,2545313276,2835853703
1016,2634427271,2532262550,5694414078
1184,2634427271,2532262550,5694414078
1305,2030648078,9101875582,5895904429
